In [24]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET

In [31]:
# Using XML web scraping from a chosen steam community page to get a large amount of varied Steam IDs
xml_url = "https://steamcommunity.com/groups/SteamDB/memberslistxml/?xml=1"

response = requests.get(xml_url)

root = ET.fromstring(response.content)

id_list = []

for x in root.iter('steamID64'):
    id_list.append(int(x.text))


In [32]:
# Set your Steam API key
api_key = "A9283666CBEBA01186FF7EB5D8F80120"

steam_ids = id_list

# List to store the dictionaries
data_list = []

# Loop through the Steam IDs
for steam_id in steam_ids:

    url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={api_key}&steamid={steam_id}&format=json"

    response = requests.get(url)

    data = json.loads(response.text)
    
    if 'games' not in data["response"]:

        print(f"Skipping {steam_id} - no games found")

        continue

    games = data["response"]['games']

    game_data = []

    for game in games:

        game_data.append({"appid": game["appid"], "playtime_forever": game["playtime_forever"]})

    dict = {'steam_id': steam_id, 'game_data': game_data}

    data_list.append(dict)


Skipping 76561197990586091 - no games found
Skipping 76561198815122240 - no games found
Skipping 76561198078811585 - no games found
Skipping 76561198112013468 - no games found
Skipping 76561198113716843 - no games found
Skipping 76561199402635936 - no games found
Skipping 76561199402670892 - no games found


In [33]:
def get_app_list():
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    response = requests.get(url)
    app_list = response.json()
    return app_list['applist']['apps']

In [34]:
game_names = get_app_list()

appid_name_map = {game['appid']: game['name'] for game in game_names}


for x in data_list:
    for game in x['game_data']:
    
        if game['appid'] in appid_name_map:
    
            game['name'] = appid_name_map[game['appid']]



In [35]:
df_data = []


for x in data_list:

    game_data = x['game_data']

    temp_data = {'steam_id': x['steam_id']}


    for game in game_data:

        if 'name' in game and 'playtime_forever' in game:

            temp_data[game['name']] = game['playtime_forever']

    
    df_data.append(temp_data)


df = pd.DataFrame(df_data)


df

,steam_id,Counter-Strike,Team Fortress Classic,Day of Defeat,Deathmatch Classic,Half-Life: Opposing Force,Ricochet,Half-Life,Half-Life: Blue Shift,Counter-Strike: Condition Zero,...,Soulslinger: Envoy of Death,Godhood,Bear and Breakfast,Slow living with Princess,Mystery Case Files: Cross Fade Collector's Edition,Mystery Case Files: Incident at Pendle Tower Collector's Edition,Boom 3D,Another Brick in The Mall,Paradiddle,City of Broken Dreamers: Book One
0,76561197972494985,161264.0,92.0,34.0,28.0,255.0,69.0,369.0,225.0,1072.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,76561198074261126,5.0,3.0,3.0,3.0,3.0,451809.0,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,76561198020275445,5.0,7.0,6.0,0.0,0.0,0.0,918.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,76561198820816240,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,76561197998401807,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,76561199020089851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
989,76561199038822713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
990,76561199384528462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,76561199446378587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Function to check for nulls and display as a table with both null values and as a percentage
def null_vals(dataframe):
    '''Function to show both number of nulls and the percentage of nulls in the whole column'''
    null_vals = dataframe.isnull().sum() # How many nulls in each column
    total_cnt = len(dataframe) # Total entries in the dataframe
    null_vals = pd.DataFrame(null_vals,columns=['null']) # Put the number of nulls in a single dataframe
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3) # Round how many nulls are there, as %, of the df
    
    return null_vals.sort_values('percent', ascending=False) #Return the number and percentage of missing values, order by percentage descending

null_vals(df) #Display the table

,null,percent
Painter Simulator,992,99.899
I saw IT,992,99.899
Trick Shot,992,99.899
Dusk Of Confinement,992,99.899
Trenchfoot,992,99.899
...,...,...
Warframe,524,52.769
Left 4 Dead 2,502,50.554
PUBG: BATTLEGROUNDS,445,44.814
Counter-Strike 2,31,3.122
